In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16339,2024-12-24,Europa Liga Aba,10:30,Dubai,Igokea,1.13,4.78,162.5,1.80,1.86,-12.5,2.05,1.65,4zQ7suHN,0.884956,0.209205,0.555556,0.537634,0.094161,140.188,60.760273,0.433420,1.8,1.643168,0.912871,104.49,1.842,1.251547,0.679450,23.0,222.342,173.228807,0.779110,1.2,1.643168,1.369306,525.60,2.784,2.178412,0.782476,-28.0,81,80,1.29,6.57,0.000,376.112,0.873414,0.023184,0.152888,0.00,0.000,inf,0.000000,0.0,0.000000,-0.64,-0.128,-29.531250,0.339322,0.3,-0.039322
16340,2024-12-24,Israel Liga Leumit,13:30,Maccabi Raanana,Hapoel Bnei Kfar Kasem,1.14,4.66,166.5,1.83,1.83,-12.5,1.95,1.72,drzNG8CL,0.877193,0.214592,0.546448,0.546448,0.091785,129.312,21.494896,0.166225,3.0,0.000000,0.000000,134.64,1.384,0.264632,0.191208,62.0,182.600,120.556935,0.660224,1.8,1.643168,0.912871,379.28,2.310,1.227110,0.531217,-50.0,99,88,1.36,4.31,112.256,224.682,0.858281,0.000000,0.088629,1.08,0.216,0.648148,0.758477,0.7,-0.058477,2.79,0.558,6.559140,0.398978,0.5,0.101022
16341,2024-12-24,Israel Liga Leumit,14:00,Elitzur Shomron,Ironi Eilat,2.41,1.47,171.5,1.83,1.83,3.5,1.95,1.72,ru8A74Z7,0.414938,0.680272,0.546448,0.546448,0.095210,234.926,58.056782,0.247128,0.6,1.341641,2.236068,239.36,2.914,0.931440,0.319643,-47.0,150.432,65.947612,0.438388,1.2,1.643168,1.369306,99.76,1.776,0.667480,0.375834,-7.0,64,86,3.74,1.16,168.220,0.000,0.342619,0.000000,0.088629,-2.38,-0.476,-2.962185,0.571559,0.5,-0.071559,0.00,0.000,inf,0.000000,0.0,0.000000
16342,2024-12-24,Israel Liga Leumit,14:00,Ramat Hasharon,Migdal Haemek,1.18,4.10,161.5,1.83,1.83,-11.5,1.97,1.71,ltF19Q4e,0.847458,0.243902,0.546448,0.546448,0.091360,136.340,43.217937,0.316986,1.8,1.643168,0.912871,82.80,1.640,0.376630,0.229652,46.0,204.972,59.508069,0.290323,0.0,0.000000,NaN,142.35,2.718,0.555401,0.204342,-42.0,60,73,1.38,1.95,150.672,213.960,0.782103,0.000000,0.099917,1.60,0.320,0.562500,0.610779,0.7,0.089221,-2.14,-0.428,-7.242991,0.000000,0.0,0.000000
16343,2024-12-24,Turquia Kbsl Feminina,12:00,Besiktas F,Bodrum F,1.66,2.05,153.5,1.83,1.83,-52.5,1.93,1.74,fBc8tzxB,0.602410,0.487805,0.546448,0.546448,0.090215,201.004,216.817028,1.078670,1.8,1.643168,0.912871,85.68,2.636,2.790462,1.058597,24.0,479.786,625.421942,1.303544,0.0,0.000000,NaN,1584.00,8.802,13.610697,1.546319,-140.0,72,48,1.19,33.00,99.228,0.000,0.148664,0.000000,0.073215,-0.21,-0.042,-15.714286,0.758923,0.8,0.041077,0.00,0.000,inf,0.000000,0.0,0.000000
16344,2024-12-24,Israel Liga Leumit,14:30,Safed,Maccabi Rehovot,3.19,1.32,154.5,1.82,1.88,7.5,1.83,1.83,W0l5TodR,0.313480,0.757576,0.549451,0.531915,0.071055,211.852,72.383536,0.341670,1.2,1.643168,1.369306,139.32,2.844,0.833864,0.293201,-22.0,116.346,14.282314,0.122757,1.2,1.643168,1.369306,99.22,1.458,0.285605,0.195888,-21.0,81,82,1.72,1.21,165.304,117.094,0.586381,0.022933,0.000000,-1.74,-0.348,-6.293103,0.495797,0.3,-0.195797,-0.77,-0.154,-2.077922,0.581551,0.6,0.018449
16345,2024-12-24,Turquia Kbsl Feminina,12:00,Botas F,Fenerbahce F,9.70,1.01,152.5,1.81,1.85,17.5,2.00,1.69,Sb6ipIxn,0.103093,0.990099,0.552486,0.540541,0.093192,153.764,37.024870,0.240790,0.6,1.341641,2.236068,123.12,2.124,0.674263,0.317449,-35.0,89.532,4.560013,0.050932,3.0,0.000000,0.000000,87.98,1.052,0.046583,0.044281,90.0,81,83,1.52,1.06,302.268,98.196,1.147480,0.015456,0.118809,-1.17,-0.234,-37.179487,0.565817,0.6

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
16343,12:00,Turquia Kbsl Feminina,Besiktas F,Bodrum F,1.66,Back Home
16356,08:35,China Cba,Beijing Royal Fighters,Xinjiang,2.03,Back Home
